In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'EgyptianDialectGender_1.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


directory = 'datasets/EgyptianDialectGender/Male'

data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            temp_data = {'text': '', 'Lang': ''}
            for line in file:
                if line.startswith('Tweet:'):
                    temp_data['text'] = line.split('Tweet:', 1)[1].strip()
                elif line.startswith('Lang:'):
                    temp_data['Lang'] = line.split('Lang:', 1)[1].strip()
                    data.append(temp_data.copy())
                    temp_data = {'text': '', 'Lang': ''}

dfm = pd.DataFrame(data)

dfm = dfm[dfm['Lang'] == 'ar']

display(dfm[:4])
display(len(dfm))

print()

directory = 'datasets/EgyptianDialectGender/Female'

data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            temp_data = {'text': '', 'Lang': ''}
            for line in file:
                if line.startswith('Tweet:'):
                    temp_data['text'] = line.split('Tweet:', 1)[1].strip()
                elif line.startswith('Lang:'):
                    temp_data['Lang'] = line.split('Lang:', 1)[1].strip()
                    data.append(temp_data.copy())
                    temp_data = {'text': '', 'Lang': ''}

dff = pd.DataFrame(data)

dff = dff[dff['Lang'] == 'ar']


display(dff[:4])
display(len(dff))

dfm = dfm[:50000]
dff = dff[:50000]

dfm['label'] = 0
dff['label'] = 1


df = pd.concat([dff, dfm], ignore_index=True)
display(len(df))
 
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

display(len(df))
display(df.columns)
display(df[:4])

classes_num = 2
display(classes_num)

ds = Dataset.from_pandas(df)

ds = ds.train_test_split(test_size=0.2)
display(ds)

max_sequence_length = 128


models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
]


for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                    
        
      

        def preprocess_function(examples):
            return tokenizer(examples['text'], truncation=True, padding="max_length",
                            max_length=max_sequence_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 8
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 64
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 200, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 200
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('EgyptianDialectGender_results_1.csv')
display(best_results)



2024-07-27 02:20:00.915269: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-27 02:20:00.940700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-27 02:20:01.310826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


,text,Lang
0,تورينو مين يا عنيا ؟ https://t.co/ED6FH8lZkO,ar
2,RT @_Kamaljr: يا ماما هتصوريني وتزعقيلي ماشى ؟ https://t.co/28tgjwbJhZ,ar
5,RT @q8_da_vinci: تم رفض توظيفي كمعلم تربيه فنيه في التربيه الخاصه. من قبل اللجنة الطبيه في مستشفى البحر في وزاره الصحه لسبب اعاقتي البصريه.…,ar
8,ده أيه الأدب و الكسوف ده أنا لو عند أخت في سن جواز ادهالك يا صاحبي 😂 https://t.co/1GPBcYlGtl,ar


224817

,text,Lang
0,"""ادْفَعْ بِالَّتِي هِيَ أَحْسَنُ فَإِذَا الَّذِي بَيْنَكَ وَبَيْنَهُ عَدَاوَةٌ كَأَنَّهُ وَلِيٌّ حَمِيمٌ""",ar
1,RT @AKettana: قاعده رقم واحد علشان تعرف تعيش و تتعامل مع الناس لازم تتعامل و تعرف و تقتنع بإن الدنيا مصالح,ar
2,سلام على من رأى عينا كادت أن تبكي فأضحكها,ar
3,RT @esraa_elbraga: من الغباء انك تشوف نفسك صح ع طول,ar


197551

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at qarib/bert-base-qarib and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
200,0.570900,0.506017,0.743250,0.743163
400,0.496400,0.483371,0.770550,0.768890
600,0.465200,0.438674,0.786350,0.785864
800,0.445000,0.431804,0.795800,0.794991
1000,0.426200,0.449502,0.790750,0.789189
1200,0.412400,0.384676,0.818500,0.818446
1400,0.306900,0.440827,0.819050,0.819048
1600,0.266800,0.489965,0.792550,0.788412
1800,0.268200,0.461142,0.819350,0.819201
2000,0.273000,0.417349,0.821700,0.821700


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.81490,0.814842
3,aubmindlab/bert-base-arabertv02-twitter,0.83235,0.832342
4,qarib/bert-base-qarib,0.82915,0.829129
